In [29]:
import gzip
import json
import pandas as pd
import numpy as np
import re

In [30]:
def open_file_df(path):
    rows = []
    with gzip.open(path, 'rt', encoding='utf-8') as f:
        for line in f:
            rows.append(json.loads(line))

    df = pd.DataFrame(rows)
    return df

In [31]:
df_rent = open_file_df('renttherunway_final_data.json.gz')
df_mod = open_file_df('modcloth_final_data.json.gz')

start with renttherunway dataset cleaning

In [34]:
# deal w string cols
str_cols = ["fit", "bust size", "rented for", "review_text","body type", "review_summary", "category"]
df_rent[str_cols] = df_rent[str_cols].astype("string")

#deal w int cols
int_cols = ["user_id", "item_id", "rating", "size", "age"]
for col in int_cols:
    df_rent[col] = pd.to_numeric(df_rent[col], errors='coerce').astype("Int64")

#weight col
df_rent["weight"] = (df_rent["weight"].astype("string").str.extract(r"(\d+)", expand=False) .astype("float"))

#height col helper function
def parse_height(x):
    if not isinstance(x, str):
        return np.nan

    # extract feet and inches
    m = re.match(r"^\s*(\d+)'\s*(\d+)?", x)
    if not m:
        return np.nan

    feet = float(m.group(1))
    inches = float(m.group(2)) if m.group(2) else 0.0

    return feet * 12 + inches

#apply height cleaning function
df_rent["height_inches"] = df_rent["height"].apply(parse_height)

#convert bust size to number(band) and letter(cup)

#band col
df_rent["bust_band"] = (df_rent["bust size"].str.extract(r"(\d+)", expand=False).astype("float"))

#cup col
df_rent["bust_cup"] = (df_rent["bust size"].str.extract(r"([a-zA-Z]+)", expand=False).str.upper())

#date col - use pd datetime
df_rent["review_date"] = pd.to_datetime(df_rent["review_date"], errors="coerce")


In [36]:
fit_dict={'fit':0, 'small':-1, 'large':1}
df_rent.fit = df_rent.fit.apply(lambda x: fit_dict[x])

In [28]:
df_rent['rented_for'] = df_rent['rented for'].replace('party: cocktail', 'party')

In [49]:
df_rent[df_rent.age>100].shape[0]/df_rent.shape[0]

0.00033758517533654644

In [76]:
# drop rows with age > 100?
df_rent = df_rent[df_rent.age <= 100]

# drop rows with age == 0?
df_rent = df_rent[df_rent.age != 0]

In [69]:
#reducing categories from 68 to 10

#10 categories definition
category_groups = {
    "dress": [
        "dress", "gown", "ballgown", "sheath", "shirtdress",
        "shift", "frock", "maxi", "mini", "midi"
    ],
    "jumpsuit/romper": [
        "jumpsuit", "romper", "overalls", "combo"
    ],
    "top": [
        "top", "blouse", "shirt", "tank", "tee", "t-shirt",
        "cami", "henley", "buttondown", "turtleneck", "blouson"
    ],
    "knitwear": [
        "sweater", "pullover", "knit", "cardigan", "crewneck",
        "hoodie", "sweatshirt", "sweatershirt"
    ],
    "pants": [
        "pants", "pant", "trouser", "trousers", "leggings",
        "legging", "jogger", "jeans", "culottes", "culotte",
        "sweatpants", "tight"
    ],
    "skirt": [
        "skirt", "skirts", "skort"
    ],
    "outerwear": [
        "jacket", "coat", "peacoat", "trench", "bomber",
        "blazer", "vest", "duster", "cape", "parka",
        "overcoat", "down"
    ],
    "ethnic/flowy": [
        "kaftan", "caftan", "kimono", "tunic", "poncho"
    ],
    "suit": ["suit"]
}

#make the map
category_map = {}
for new_cat, old_list in category_groups.items():
    for old_cat in old_list:
        category_map[old_cat] = new_cat

#apply mapping and set 'other' for unmapped
df_rent["category_clean"] = df_rent["category"].map(category_map).fillna("misc")


/var/folders/t4/vmyf_nk5583876h6ww663db40000gn/T/ipykernel_42383/1918013363.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rent["category_clean"] = df_rent["category"].map(category_map).fillna("misc")


In [ ]:
#review text

In [59]:
#do this during pre processing for the model

cup_order = ["AA", "A", "B", "C", "D", "DD", "DDD", "E", "F", "G"]
#df_rent["bust_cup"] = pd.Categorical(df_rent["bust_cup"],categories=cup_order,ordered=True)

now cleaning modcloth dataset